In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!kaggle datasets download -d orvile/mias-mamography-splitted-dataset -p /content/drive/MyDrive/mias
!unzip /content/drive/MyDrive/mias/*.zip -d /content/drive/MyDrive/mias
